In [45]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [46]:
text_data_sentence = pd.read_csv('textdatanew.csv', encoding='ISO-8859-1')

In [47]:
text_data_sentence.head()

,CollectionID,BiographyID,ParagraphNo,ParagraphText
0,a001,bio04,1,"A FRENCH philosopher, moralizing on the great ..."
1,a001,bio04,2,Cleopatra was joint heir to the throne of Egyp...
2,a001,bio04,3,Cleopatra might have responded with a brillian...
3,a001,bio04,4,Caesar was then above fifty years of age. His ...
4,a001,bio04,5,For three years Cleopatra reigned with little ...


## Reading Response File

In [48]:
bess_tags = pd.read_csv('CBW_Bess_tags_final2.csv')

In [49]:
bess_tags.head()

,Content,Event,Type,para no,URI,author,biographyID,collectionID,personaName,title
0,after,name,stageOfLife,1.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
1,culmination,name,stageOfLife,1.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
2,middle,name,stageOfLife,2.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
3,middle,name,stageOfLife,3.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...
4,middle,name,stageOfLife,4.0,a001.bio04.bess.xml,Willis John Abbot,bio04,a001,Cleopatra,Cleopatra (B.C. 69-30): The World's Most Famou...


In [50]:
bess_reponse = bess_tags.loc[:,['Content','Event','Type','para no','biographyID','collectionID']]

In [51]:
bess_reponse= bess_reponse.fillna(' ')

In [52]:
bess_reponse.loc[:,'Response'] = bess_reponse.loc[:,['Content','Event']].apply(lambda x: '_'.join(x),axis = 1)

In [53]:
bess_reponse.Type.value_counts()

Event                 79270
discourse             43549
topos                 19366
stageOfLife           15749
personaDescription    12710
                         19
Name: Type, dtype: int64

In [54]:
top_10_discourse = bess_reponse[bess_reponse.Type.isin(['discourse'])].Response.value_counts()[0:10]
top_10_discourse


evaluation_type                                          2320
emphasis in typeface, punctuation_type                   2165
quotation, agent's speech, unique_type                   1685
present tense_type                                       1667
description of agent or identified set of agents_type    1572
direct address, use of we_type                           1517
focalization, attributing feelings_type                  1175
quotation, persona speech, unique_type                   1120
speech, reported not quoted_type                         1096
quotation, unidentified_type                              976
Name: Response, dtype: int64

In [55]:
bess_reponse.shape

(170663, 7)

In [56]:
bess_response_df = bess_reponse.loc[:,['para no','biographyID','collectionID','Response','Type']].drop_duplicates()

In [57]:
final_response_file = bess_response_df[bess_response_df.Response.isin(top_10_discourse.index)]

In [58]:
final_response_file.head()

,para no,biographyID,collectionID,Response,Type
112,1,bio04,a001,"speech, reported not quoted_type",discourse
121,3,bio04,a001,evaluation_type,discourse
124,4,bio04,a001,description of agent or identified set of agen...,discourse
126,4,bio04,a001,"focalization, attributing feelings_type",discourse
132,5,bio04,a001,"quotation, persona speech, unique_type",discourse


In [59]:
final_response_file.loc[:,'values'] = 1

/Applications/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Applications/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [60]:
final_response_wide = (final_response_file.pivot_table(index=['para no','biographyID','collectionID','Type'],\
                                                       columns='Response', values='values').reset_index())

In [61]:
final_response_wide.head()

Response,para no,biographyID,collectionID,Type,description of agent or identified set of agents_type,"direct address, use of we_type","emphasis in typeface, punctuation_type",evaluation_type,"focalization, attributing feelings_type",present tense_type,"quotation, agent's speech, unique_type","quotation, persona speech, unique_type","quotation, unidentified_type","speech, reported not quoted_type"
0,1.0,bio01,a160,discourse,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,bio01,a162,discourse,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,bio01,a186,discourse,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,bio01,a207,discourse,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,bio01,a221,discourse,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [62]:
final_response_wide=final_response_wide.fillna(0)

In [63]:
text_data_sentence_final = pd.merge(text_data_sentence,final_response_wide, how = 'left', left_on = ['CollectionID','BiographyID','ParagraphNo'],\
                             right_on = ['collectionID','biographyID','para no'])

/Applications/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:969: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)


In [64]:
text_data_sentence_final = text_data_sentence_final[~text_data_sentence_final['para no'].isna()]

In [65]:
text_data_sentence_final_response = text_data_sentence_final[text_data_sentence_final.columns[-10:]]

In [66]:
final_response_file.Response.value_counts()

evaluation_type                                          2286
emphasis in typeface, punctuation_type                   2140
present tense_type                                       1645
quotation, agent's speech, unique_type                   1606
description of agent or identified set of agents_type    1535
direct address, use of we_type                           1481
focalization, attributing feelings_type                  1169
speech, reported not quoted_type                         1078
quotation, persona speech, unique_type                   1077
quotation, unidentified_type                              950
Name: Response, dtype: int64

### Bag of words


In [67]:
stop_words = stopwords.words('english')

In [68]:
bow_model = CountVectorizer(ngram_range= (1,2),stop_words=stop_words)

In [69]:
Para_text_bow = bow_model.fit_transform(text_data_sentence_final.ParagraphText)

### TFIDF

In [28]:
tf_idf_model = TfidfVectorizer(ngram_range=(1,2),stop_words = stop_words)

In [29]:
Para_text_tfidf = tf_idf_model.fit_transform(text_data_sentence_final.ParagraphText)

### Splitting data into train and test 

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(Para_text_tfidf,text_data_sentence_final_response, \
                                                    test_size = 0.3, random_state = 0)

In [34]:
y_test.shape

(2645, 10)

### ML models

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [36]:
#knn_model = KNeighborsClassifier(n_neighbors= 3, p = 1.5)
rf_model = RandomForestClassifier()
lr_model = LogisticRegression(multi_class='multinomial')

In [37]:
rf_model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [39]:
feature_importances = pd.DataFrame(rf_model.feature_importances_,
                                  index = bow_model.get_feature_names(),
                                  columns=['importance']).sort_values('importance',ascending=False)

feature_importances

,importance
us,0.005080
said,0.002402
one,0.001669
felt,0.001260
king,0.001247
would,0.001204
life,0.001110
thou,0.001068
sister,0.001027
asked,0.000995


### Getting additional features

In [70]:
text_features = pd.read_csv('text_features.csv', encoding='ISO-8859-1')

/Applications/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11,12,15,16,22,23,24,29,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
text_features.head()

,CollectionID,BiographyID,ParagraphNo,sadness,joy,fear,disgust,anger,score,sentiment,...,Number,Organization,Person,PrintMedia,Quantity,Sport,SportingEvent,TelevisionShow,Time,Vehicle
0,a001,bio04,1,0.255896,0.558011,0.101166,0.111615,0.054668,0.290669,positive,...,NaN,NaN,Cleopatra,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a001,bio04,2,0.171629,0.257088,0.173474,0.098726,0.267978,0.000000,neutral,...,NaN,Roman senate,"Cleopatra, Julius Caesar, Pompey, Ptolemy",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a001,bio04,3,0.247987,0.632947,0.121953,0.075352,0.082463,0.376424,positive,...,NaN,NaN,"Apollodorus, Caesar, Cleopatra",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,a001,bio04,4,0.445363,0.665001,0.158016,0.093975,0.070956,-0.077556,negative,...,NaN,NaN,"Brutus, Caesar, Cleopatra, Julius, Ptolemy",NaN,fifty years,NaN,NaN,NaN,NaN,NaN
4,a001,bio04,5,0.541249,0.589288,0.093628,0.063582,0.138158,0.452558,positive,...,NaN,NaN,"Brutus, Caesar, Cassius, Cleopatra, Mark Anton...",NaN,three years,NaN,NaN,NaN,NaN,NaN


In [75]:
list(text_features)

['CollectionID',
 'BiographyID',
 'ParagraphNo',
 'sadness',
 'joy',
 'fear',
 'disgust',
 'anger',
 'score',
 'sentiment',
 'ParagraphText',
 'Anatomy',
 'Award',
 'Company',
 'Crime',
 'Date',
 'Drug',
 'Facility',
 'GeographicFeature',
 'HealthCondition',
 'JobTitle',
 'Location',
 'Measure',
 'Movie',
 'Number',
 'Organization',
 'Person',
 'PrintMedia',
 'Quantity',
 'Sport',
 'SportingEvent',
 'TelevisionShow',
 'Time',
 'Vehicle']

In [81]:
text_data_sentence_final.drop(['para no'] , axis='columns', inplace=True)

In [82]:
text_data_sentence_final

,CollectionID,BiographyID,ParagraphNo,ParagraphText,Type,description of agent or identified set of agents_type,"direct address, use of we_type","emphasis in typeface, punctuation_type",evaluation_type,"focalization, attributing feelings_type",present tense_type,"quotation, agent's speech, unique_type","quotation, persona speech, unique_type","quotation, unidentified_type","speech, reported not quoted_type"
0,a001,bio04,1,"A FRENCH philosopher, moralizing on the great ...",discourse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,a001,bio04,3,Cleopatra might have responded with a brillian...,discourse,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,a001,bio04,4,Caesar was then above fifty years of age. His ...,discourse,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,a001,bio04,5,For three years Cleopatra reigned with little ...,discourse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,a001,bio04,6,For Antony now were all the amorous dalliances...,discourse,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,a001,bio04,8,Cleopatra took refuge in a massive mausoleum w...,discourse,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,a001,bio04,9,But Octavius was cold and ambitious. His desir...,discourse,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,a001,bio20,1,STEWING in a great copper bathtub shaped like ...,discourse,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,a001,bio20,2,"Scenting more blood, the ogre of the French re...",discourse,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11,a001,bio20,3,The woman who entered was about twenty-four ye...,discourse,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
text_data_feat = pd.merge(text_data_sentence_final,text_features, how = 'left', left_on = ['CollectionID','BiographyID','ParagraphNo', 'ParagraphText'],\
                             right_on = ['CollectionID','BiographyID','ParagraphNo', 'ParagraphText'])

In [95]:
text_data_feat.Type.value_counts()

discourse    8814
topos           1
Name: Type, dtype: int64

In [87]:
list(text_data_feat)

['CollectionID',
 'BiographyID',
 'ParagraphNo',
 'ParagraphText',
 'Type',
 'description of agent or identified set of agents_type',
 'direct address, use of we_type',
 'emphasis in typeface, punctuation_type',
 'evaluation_type',
 'focalization, attributing feelings_type',
 'present tense_type',
 "quotation, agent's speech, unique_type",
 'quotation, persona speech, unique_type',
 'quotation, unidentified_type',
 'speech, reported not quoted_type',
 'sadness',
 'joy',
 'fear',
 'disgust',
 'anger',
 'score',
 'sentiment',
 'Anatomy',
 'Award',
 'Company',
 'Crime',
 'Date',
 'Drug',
 'Facility',
 'GeographicFeature',
 'HealthCondition',
 'JobTitle',
 'Location',
 'Measure',
 'Movie',
 'Number',
 'Organization',
 'Person',
 'PrintMedia',
 'Quantity',
 'Sport',
 'SportingEvent',
 'TelevisionShow',
 'Time',
 'Vehicle']

In [94]:
bess_tags.Type.value_counts()

Event                 79270
discourse             43549
topos                 19366
stageOfLife           15749
personaDescription    12710
Name: Type, dtype: int64

In [102]:
response_feat = text_data_feat[text_data_feat.columns[]]

,CollectionID,BiographyID,ParagraphNo,ParagraphText,Type,description of agent or identified set of agents_type,"direct address, use of we_type","emphasis in typeface, punctuation_type",evaluation_type,"focalization, attributing feelings_type",...,Number,Organization,Person,PrintMedia,Quantity,Sport,SportingEvent,TelevisionShow,Time,Vehicle
0,a001,bio04,1,"A FRENCH philosopher, moralizing on the great ...",discourse,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,Cleopatra,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a001,bio04,3,Cleopatra might have responded with a brillian...,discourse,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,"Apollodorus, Caesar, Cleopatra",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a001,bio04,4,Caesar was then above fifty years of age. His ...,discourse,1.0,0.0,0.0,0.0,1.0,...,NaN,NaN,"Brutus, Caesar, Cleopatra, Julius, Ptolemy",NaN,fifty years,NaN,NaN,NaN,NaN,NaN
3,a001,bio04,5,For three years Cleopatra reigned with little ...,discourse,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,"Brutus, Caesar, Cassius, Cleopatra, Mark Anton...",NaN,three years,NaN,NaN,NaN,NaN,NaN
4,a001,bio04,6,For Antony now were all the amorous dalliances...,discourse,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,"Antony, Caesar, Cleopatra",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
X_train, X_test, y_train, y_test = train_test_split(text_data_feat,text_data_sentence_final_response, \
                                                    test_size = 0.3, random_state = 0)

In [103]:
from xgboost import XGBClassifier

In [101]:
model = XGBClassifier()
model.fit(X_train, y_train)

ValueError: bad input shape (6170, 10)